In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
from sklearn import metrics



In [ ]:

# Load data
weekly_data = pd.read_csv(r'C:\Users\joshu\OneDrive\Desktop\CS74\Final_Project\music_econ_merged.csv')


# Prepare features

audio_features = ['danceability', 'energy', 'valence', 'tempo',  'acousticness', 'instrumentalness', 'speechiness',  'loudness', 'pos', 'neg']

# Economic features (if you want to include them)
econ_features = ['UNRATE', 'CPIAUCSL']

# Topic features (columns '0' through '29')
topic_features = [str(i) for i in range(30)]



X = weekly_data[['danceability', 'energy', 'valence', 'tempo', 
                 'acousticness', 'instrumentalness', 'speechiness', 
                 'loudness', 'pos', 'neg']]

y = weekly_data['USREC']

print(f"\nTotal data: {len(X)} observations")
print(f"Total recessions: {y.sum()} ({y.mean()*100:.1f}%)")
print(f"Total normal periods: {(y==0).sum()} ({(y==0).mean()*100:.1f}%)")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,       random_state=42,      stratify=y           
)

print(f"Total: {len(X_train)} observations")
print(f"Recessions: {y_train.sum()} ({y_train.mean()*100:.1f}%)")
print(f"Normal: {(y_train==0).sum()} ({(y_train==0).mean()*100:.1f}%)")

print(f"\n--- TEST SET ---")
print(f"Total: {len(X_test)} observations")
print(f"Recessions: {y_test.sum()} ({y_test.mean()*100:.1f}%)")
print(f"Normal: {(y_test==0).sum()} ({(y_test==0).mean()*100:.1f}%)")

log_reg = LogisticRegression(
    class_weight='balanced',  # Handles class imbalance
    max_iter=1000,           # Enough iterations to converge
    random_state=42
)
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
baseline_accuracy = (y_test == 0).mean()  # Always predicting "no recession"


print(f"\n🔹 Baseline (always predict no recession): {baseline_accuracy:.3f}")
print(f"🔹 Logistic Regression Accuracy:          {accuracy:.3f}")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

tn, fp, fn, tp = cm.ravel()
print(f"\nTrue Negatives (correctly predicted normal): {tn}")
print(f"False Positives (predicted recession but normal): {fp}")
print(f"False Negatives (missed recession): {fn}")
print(f"True Positives (correctly predicted recession): {tp}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))



coeff_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': log_reg.coef_[0]
}).sort_values('Coefficient', ascending=False)

print(coeff_df.to_string(index=False))

top_pos = coeff_df.head(3)['Feature'].tolist()
top_neg = coeff_df.tail(3)['Feature'].tolist()

print("\nMost positive predictors of recession:", ", ".join(top_pos))
print("Most negative predictors of recession:", ", ".join(top_neg))

print("\n✅ Done! Model training and evaluation complete.")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix
class_names=['No Recession', 'Recession']
fig, ax = plt.subplots(figsize=(6, 6))
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')



In [ ]:



# my labes for interpretation
topic_labels = {
    '0': "Spiritual", '1': "Relationship", '2': "Longing",
    '3': "Clarity", '4': "Everyday Joy", '5': "Dance/Party",
    '6': "Heartbreak", '7': "Intimacy", '8': "Self/Body",
    '9': "Party/Club", '10': "Bop/Dance", '11': "Social Talk",
    '12': "Girl & Fun", '13': "Music & Friends", '14': "Time & Change",
    '15': "Appreciation", '16': "Admiring", '17': "Regret",
    '18': "Past/Memory", '19': "Hopes & Dreams", '20': "True Love",
    '21': "Money/Power", '22': "Romantic", '23': "Hip-Hop",
    '24': "Home", '25': "Dance & Music", '26': "Playfulness",
    '27': "Feeling Good", '28': "Freaky-Deaky", '29': "Yearning"
}

# Create coefficient dataframe
coeff_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': log_reg.coef_[0]
})

# Add labels and category
coeff_df['Label'] = coeff_df['Feature'].map(topic_labels).fillna('')  # ← FIXED: Use empty string instead of NaN
coeff_df['Category'] = coeff_df['Feature'].apply(
    lambda x: 'Topic' if x in topic_labels 
    else 'Economic' if x in econ_features 
    else 'Audio/Sentiment'
)

# Sort by absolute coefficient
coeff_df['Abs_Coef'] = coeff_df['Coefficient'].abs()
coeff_df = coeff_df.sort_values('Abs_Coef', ascending=False)

print("\nTop 20 Most Important Features:")
for i, row in coeff_df.head(20).iterrows():
    feature = row['Feature']
    coef = row['Coefficient']
    category = row['Category']
    label = row['Label']
    
    if label:  # ← FIXED: Now label is empty string instead of NaN
        print(f"  {feature:15s} [{category:15s}] ({label:20s}): {coef:+.4f}")
    else:
        print(f"  {feature:15s} [{category:15s}]: {coef:+.4f}")

# Separate analysis by category


for category in ['Audio/Sentiment', 'Economic', 'Topic']:
    cat_df = coeff_df[coeff_df['Category'] == category].head(5)
    
    if len(cat_df) > 0:
        print(f"\n{category}:")
        for i, row in cat_df.iterrows():
            feature = row['Feature']
            coef = row['Coefficient']
            label = row['Label']
            label_str = f" ({label})" if label else ""  
            print(f"  {feature:15s}{label_str:22s}: {coef:+.4f}")



top_recession_predictors = coeff_df.nlargest(5, 'Coefficient')
top_normal_predictors = coeff_df.nsmallest(5, 'Coefficient')

print("\nTop 5 Predictors of RECESSION (+):")
for i, row in top_recession_predictors.iterrows():
    label = row['Label']
    label_str = f" ({label})" if label else "" 
    print(f"  {row['Feature']}{label_str}: {row['Coefficient']:+.4f}")

print("\nTop 5 Predictors of NORMAL (-):")
for i, row in top_normal_predictors.iterrows():
    label = row['Label']
    label_str = f" ({label})" if label else "" 
    print(f"  {row['Feature']}{label_str}: {row['Coefficient']:+.4f}")



print("\nPrevious (Audio + Econ only):")
print("  Recall: 86.7%")
print("  Precision: 31.7%")
print("  False Positives: 28")

print(f"\nCurrent (Audio + Econ + Topics):")
print(f"  Recall: {recall*100:.1f}%")
print(f"  Precision: {precision*100:.1f}%")
print(f"  False Positives: {fp}")

# Calculate improvement
recall_change = (recall - 0.867) * 100
precision_change = (precision - 0.317) * 100
fp_change = fp - 28

print(f"\nChanges:")
print(f"  Recall: {recall_change:+.1f} percentage points")
print(f"  Precision: {precision_change:+.1f} percentage points")
print(f"  False Positives: {fp_change:+d}")

if recall >= 0.867:
    print("\n✅ Topics IMPROVED recall!")
elif recall >= 0.85:
    print("\n⚠️  Topics maintained similar recall")
else:
    print("\n⚠️  Topics decreased recall (possible overfitting)")

if precision > 0.317:
    print("✅ Topics IMPROVED precision (fewer false alarms)!")

print("\n✅ Done! Model with topics training and evaluation complete.")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix



df = pd.read_csv(r'C:\Users\joshu\OneDrive\Desktop\CS74\Final_Project\music_econ_topics_merged.csv')

# Define feature sets
audio_features = ['danceability', 'energy', 'valence', 'tempo', 
                  'acousticness', 'instrumentalness', 'speechiness', 
                  'loudness', 'pos', 'neg']

econ_features = ['UNRATE', 'CPIAUCSL']
topic_features = [str(i) for i in range(30)]


models_to_test = {
    '1. Music Only (MAIN)': audio_features,
    '2. Music + Economics': audio_features + econ_features,
    '3. Music + Topics': audio_features + topic_features,
    '4. Music + Economics + Topics (FULL)': audio_features + econ_features + topic_features
}

results = []

for model_name, features in models_to_test.items():
    print(f"\n{'='*70}")
    print(f"{model_name}")
    print(f"{'='*70}")
    
    # Check available features
    available = [f for f in features if f in df.columns]
    
    if len(available) == 0:
        print("⚠️  No features available, skipping")
        continue
    
    print(f"Features: {len(available)}")
    
    # Prepare data
    df_clean = df[available + ['USREC']].dropna()
    X = df_clean[available]
    y = df_clean['USREC']
    
    print(f"Observations: {len(X)}")
    
    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Train
    model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Metrics
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # Store
    results.append({
        'Model': model_name,
        'Features': len(available),
        'Accuracy': accuracy,
        'Recall': recall,
        'Precision': precision,
        'False Positives': fp,
        'True Positives': tp,
        'Total Recessions': tp + fn
    })
    
    print(f"\nRecall: {recall:.1%} ({tp}/{tp+fn} recessions caught)")
    print(f"Precision: {precision:.1%}")
    print(f"False Positives: {fp}")


import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df[['Model', 'Features', 'Recall', 'Precision', 'False Positives']]

# Format as percentages
results_df['Recall'] = results_df['Recall'].apply(lambda x: f"{x:.1%}")
results_df['Precision'] = results_df['Precision'].apply(lambda x: f"{x:.1%}")

print("\n" + results_df.to_string(index=False))



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

models = ['Music\nOnly', 'Music +\nEconomics', 'Music +\nTopics', 'Full\nModel']
recall = [86.7, 86.7, 86.7, 86.7]
precision = [30.2, 31.7, 31.0, 32.5]
features = [10, 12, 40, 42]

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(models))
width = 0.35

bars1 = ax.bar(x - width/2, recall, width, label='Recall', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, precision, width, label='Precision', color='coral', alpha=0.8)

ax.set_xlabel('Model', fontweight='bold', fontsize=12)
ax.set_ylabel('Percentage (%)', fontweight='bold', fontsize=12)
ax.set_title('Model Performance: Recall vs Precision', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim([0, 100])
ax.axhline(y=86.7, color='steelblue', linestyle='--', alpha=0.5, linewidth=1)
ax.grid(axis='y', alpha=0.3)

# Add feature count labels
for i, (model, feat) in enumerate(zip(models, features)):
    ax.text(i, 5, f'{feat} features', ha='center', fontsize=9, style='italic')

plt.tight_layout()
plt.savefig(r'C:\Users\joshu\OneDrive\Desktop\CS74\Final_Project\model_comparison_final.png', 
            dpi=300, bbox_inches='tight')
print("✓ Saved chart")
plt.show()